In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 하이퍼 파라미터 지정
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 데이타셋 로딩 -- 2번에 걸쳐서 로딩
# train_dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = True, 
                                           transform=transforms.ToTensor(),
                                           download = True)
test_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train = False, 
                                           transform=transforms.ToTensor())

# 2step...DataLoader...BatchSize를 이용함
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

Model 생성하기


In [ ]:
class NeuralNet(nn.Module):
    # 모델 설계
    def __init__(self, input_size,  hidden_size  ,num_classes):

        super(NeuralNet, self).__init__()
        self.fc1=nn.Linear(input_size, hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size, num_classes)

    # 모델의 Forward Path를 정의
    def forward(self, x):
        out=self.fc1(x)        
        out=self.relu(out)
        out=self.fc2(out)

        return out  # 여기까지가 클래스 정의부분

# 위에서 정의한 클래스를 인스턴스화 시킴
model = NeuralNet(input_size, hidden_size, num_classes).to(device) # 이 model 을 gpu서버에서 돌리겠다

# loss and optimizer를 선정의
loss_function = nn.CrossEntropyLoss() # Loss 기능안에 Softmax 함수기능 포함되어져 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step=len(train_loader) # 600번 로딩..600번 학습이 진행된다.

for epoch in range(num_epochs): # 5번 ...
    for i, (images, labels) in enumerate(train_loader): 
        # 네트워크에 넣어줄때는 1차원으로 펼쳐서 넣는다
        images = images.reshape(-1, 28*28).to(device) #gpu로 돌린다.
        labels = labels.to(device)

        # Forward Pass
        pred=model(images)
        loss = loss_function(pred, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if(i+1) % 100 ==0:
            print('Epoch[{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))


